<a href="https://colab.research.google.com/github/Megha21337/Machine-Learning-Assignment/blob/main/SeeYou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
data=pd.read_csv('/content/drive/My Drive/abcd/Reviews.csv')
col=['Summary','Text']
data[col].to_csv('Modified_review.csv',index=False)

In [3]:
del data

In [4]:
df=pd.read_csv('/content/drive/My Drive/abcd/Modified_review.csv')
rows=df[df['Summary'].isna() | df['Text'].isna()].shape[0]
print(rows)

27


In [5]:
df.dropna(subset=['Summary','Text'],inplace=True)

In [6]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import string

In [8]:
def html_remove(text):
    word=BeautifulSoup(text,"html.parser")
    return word.get_text()

In [9]:
def accented(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [10]:
def preprocess(text):
    text = html_remove(text)
    text = accented(text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [word for word in tokens if word not in string.punctuation]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(lemmatized_tokens)

df['Summary'] = df['Summary'].apply(preprocess)
df['Text'] = df['Text'].apply(preprocess)

df.to_csv('Preprocessed_Review1.csv', index=False)


<ipython-input-8-262890ab649b>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  word=BeautifulSoup(text,"html.parser")
<ipython-input-8-262890ab649b>:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  word=BeautifulSoup(text,"html.parser")
<ipython-input-8-262890ab649b>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  word=BeautifulSoup(text,"html.parser")


In [11]:
df = pd.read_csv('Preprocessed_Review1.csv')
sampled_df = df.sample(frac=0.30, random_state=42)

sampled_df.to_csv('/content/drive/My Drive/abcd/less_data.csv', index=False)


In [12]:
import sklearn
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(sampled_df, test_size=0.25, random_state=42)

train_df.to_csv('/content/drive/My Drive/abcd/train.csv', index=False)
test_df.to_csv('/content/drive/My Drive/abcd/test.csv', index=False)

In [13]:
n1=train_df.shape[0]
n2=test_df.shape[0]
print(n1)
print(n2)

127896
42632


In [14]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [15]:

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


In [ ]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [ ]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()